<b>Google Colab</b> <a href="https://colab.research.google.com/github/kirillzyusko/deeplearning/blob/master/4/lab.ipynb">link</a>

# **Part 1: Download dataset, extract, split, check data distribution**

In [6]:
import os
import urllib.request

!wget "http://ufldl.stanford.edu/housenumbers/train_32x32.mat"
!wget "http://ufldl.stanford.edu/housenumbers/test_32x32.mat"
!wget "http://ufldl.stanford.edu/housenumbers/extra_32x32.mat"

%ls

--2020-02-24 13:02:42--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat.2’

train_32x32.mat.2   100%[===================>] 173.61M  19.8MB/s    in 12s     

2020-02-24 13:02:55 (14.4 MB/s) - ‘train_32x32.mat.2’ saved [182040794/182040794]

--2020-02-24 13:02:56--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat.1’

test_32x32.mat.1    100%[===================>]  61.30M  16.0MB/s    in 6.2s    

2020-02-24 13:03:02 (9.89 MB/s) - ‘test_32x32.mat.1’ saved [642

Extract .zip

In [0]:
%ls

adc.json  data/  kaggle/  sample_data/


# **Part 2: DNN model**

# **Part 3: Applying data augmentation**

# **Part 4: Using transfer learning**